# Pair Programming Preprocesado:

### Usando el mismo dataset que usatéis ayer, los objetivos de los ejercicios de hoy son:


In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd


# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


# Estandarización variables numéricas y Codificación variables categóricas
# ------------------------------------------------------------------------------
from sklearn.preprocessing import StandardScaler

# Gestión datos desbalanceados
# ------------------------------------------------------------------------------
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek

# Para separar los datos en train y test
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split

#  Gestión de warnings
# ------------------------------------------------------------------------------
import warnings
warnings.filterwarnings("ignore")

In [2]:
#abrimos el CSV:

df = pd.read_csv('datos/Social_Network_Ads_1.csv')
df.head()

,user_id,gender,age,estimatedsalary,purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


- #### Estandarizar las variables numéricas de vuestro set de datos


In [3]:
#iniciamos el método:

scaler = StandardScaler()

In [4]:
#extraemos las columnas numéricas del df:
numericas = df.select_dtypes(include=np.number)
numericas.head()


,user_id,age,estimatedsalary,purchased
0,15624510,19,19000,0
1,15810944,35,20000,0
2,15668575,26,43000,0
3,15603246,27,57000,0
4,15804002,19,76000,0


Observamos que en la variable user_id (aquella que incluye los identificadores de cada caso) se van a estandarizar los datos, por lo que procedemos a eliminar la columna.

In [5]:
#eliminamos del df original:

df.drop('user_id', axis=1, inplace=True)
df.head()

,gender,age,estimatedsalary,purchased
0,Male,19,19000,0
1,Male,35,20000,0
2,Female,26,43000,0
3,Female,27,57000,0
4,Male,19,76000,0


- #### Codificar las variables categóricas. Recordad que tendréis que tener en cuenta si vuestras variables tienen orden o no.


Antes de unir los dfs, procedemos a recodificar las variables categóricas, ya que de lo contrario en los valores estandarizados obtenidos en 'numericas_est', nos aparecería el género como NaN.

In [6]:
categorica = df.select_dtypes(include='object')
categorica.head()

,gender
0,Male
1,Male
2,Female
3,Female
4,Male


Nuestras variables no tienen orden, por lo que utilizaremos GetDummies para recodificar la variable de género.

In [7]:
df_codif = pd.DataFrame() #para transformar a df

for col in categorica.columns: #iteramos por las columnas para generar una por cada valor:
    df_dummies = pd.get_dummies(df[col],prefix = col, prefix_sep = "_", dtype = int)
    #concatenamos:
    df_codif = pd.concat([df_codif, df_dummies], axis = 1)
    
#comprobamos el cambio
df_codif.head(3)

,gender_Female,gender_Male
0,0,1
1,0,1
2,1,0


In [8]:
#hacemos un for loop para insertar las nuevas columnas en el df original:
for x in df_codif.columns:
    df[x] = df_codif[x]

df.drop('gender', axis=1, inplace=True) #borramos la columna de género original

#comprobamos que se han realizado los cambios:
df.columns

Index(['age', 'estimatedsalary', 'purchased', 'gender_Female', 'gender_Male'], dtype='object')

#### Estandarizamos:

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   age              400 non-null    int64
 1   estimatedsalary  400 non-null    int64
 2   purchased        400 non-null    int64
 3   gender_Female    400 non-null    int64
 4   gender_Male      400 non-null    int64
dtypes: int64(5)
memory usage: 15.8 KB


Vemos que ahora todas las columnas son int (numéricas).

In [10]:
#volvemos a definir el df con las variables numéricas después de recodificar la variable de género:
numericas = df.select_dtypes(include=np.number)
numericas.head()

,age,estimatedsalary,purchased,gender_Female,gender_Male
0,19,19000,0,0,1
1,35,20000,0,0,1
2,26,43000,0,1,0
3,27,57000,0,1,0
4,19,76000,0,0,1


In [11]:
#ajustamos nuestros datos al modelo:
scaler.fit(numericas)

# transformamos dichos datos:
X_esc = scaler.transform(numericas)

# los convertimos en un df:

numericas_est = pd.DataFrame(X_esc, columns = numericas.columns)
numericas_est.head(2)

,age,estimatedsalary,purchased,gender_Female,gender_Male
0,-1.781797,-1.490046,-0.745936,-1.020204,1.020204
1,-0.253587,-1.460681,-0.745936,-1.020204,1.020204


In [12]:
#vamos a unir los cambios en numericas_est al df original:

df_fusion = pd.concat([df, numericas_est])

df_fusion.sample(3) #sample para que nos devuelva casos de ambos dfs

,age,estimatedsalary,purchased,gender_Female,gender_Male
24,46.000000,23000.000000,1.000000,0.000000,1.000000
247,1.847701,1.534534,1.340598,0.980196,-0.980196
4,19.000000,76000.000000,0.000000,0.000000,1.000000


### 

- #### Chequead si vuestros datos están balanceados. En caso de que no lo estén utilizad algunas de las herramientas aprendidas en la lección para balancearlos.


Vamos a comprobar si nuestros datos están balanceados o no:

In [13]:
#separamos vd/vr del df:
y = df['purchased']
X = df.drop('purchased', axis=1)

#dividimos en sets de entrenamiento y test:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7)

In [21]:
# iniciamos el método
os_us = SMOTETomek()

# ajustamos el modelo
X_train_res, y_train_res = os_us.fit_resample(X_train, y_train)

#vamos a comprobar los cambios en la distribución de la columna:
print(f"Antes del resample, la distribución era: \n{y_train.value_counts()}")
print("___________________________________________________________________")
print(f"Después del resample, la distribución es: \n{y_train_res.value_counts()}")

Antes del resample, la distribución era: 
0    182
1     98
Name: purchased, dtype: int64
___________________________________________________________________
Después del resample, la distribución es: 
0    167
1    167
Name: purchased, dtype: int64


In [18]:
df_resample = (x_train, y_train_res)

0      0
1      0
2      0
3      0
4      0
      ..
319    1
320    1
321    1
322    1
323    1
Name: purchased, Length: 324, dtype: int64

Vemos que nuestros datos ahora están balanceados.

- #### Guardad el dataframe con los cambios que habéis aplicado para utilizarlo en la siguiente lección.

In [15]:
#guardamos el df fusionado
df_fusion.to_csv('datos/EST_Social_Network_Ads.csv')